# We want to create a synthetic time series data set

> Based on the synthetic data generation notebook of the ```afa``` repository by Henrik von Kleist.

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import pathlib
relative_path_to_afa = '../../afa_github/afa'
#sys.path.insert(0, os.path.abspath(relative_path_to_afa))
sys.path.insert(1, os.path.join(sys.path[0], relative_path_to_afa))
print('CWD:', pathlib.Path(pathlib.Path.cwd()))

import sklearn

# data generation
from afa_datasets.synthetic_data.data_generation_ts import generate_data_ts

# loading/ saving
from data_modelling.clairvoyance_static.datasets.data_loader import CSVLoader, save_dataset
from data_modelling.clairvoyance_static.utils.model_utils import PipelineComposer

CWD: /home2/joshua.wendland/Documents/sepsis/toy_dataset


2022-11-28 12:47:56.559341: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-28 12:47:56.559367: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
# define directories
data_dir = "dataset/"
dataset_name = "synthetic_1"

# Datageneration

Use sklearn's make-classification function

In [26]:
# df_train, df_test = generate_data_ts(data_dir = data_dir + dataset_name + "/", 
#                                          dataset_name = dataset_name,
#                                          n_features_static  = 4 ,
#                                          n_features_ts  = 10 ,
#                                          n_redundant = 0 ,
#                                          n_informative = 2,
#                                          n_datapts   = 200000,
#                                          n_classes   = 2,
#                                          random_state  = 47,
#                                          test_size   = 0.5)

data_train_ts_df, data_train_static_df, data_test_ts_df, data_test_static_df = generate_data_ts(data_dir = "",
                                        dataset_name = "synthetic_ts_1",
                                        n_features_static  = 2  ,
                                        n_features_ts      = 4  , 
                                        n_timepts   = 50,
                                        n_datapts   = 100, 
                                        test_size = 0.1,
                                        table_style = 'wide',
                                        n_classes   = 2)

# Load the Data

First import necessary stuff.

In [2]:
import pandas as pd

Load the data and show the beginning.

In [3]:
path = '/home2/joshua.wendland/Documents/sepsis/toy_dataset/synthetic_ts_1/synthetic_ts_test_data_eav.csv.gz'

df = pd.read_csv(path, compression=None)
df = df.sort_values(by=['id', 'time'], ascending=True, ignore_index=True)  # time was not sorted

print(df.head)


<bound method NDFrame.head of         id  time    X_ts_0    X_ts_1    X_ts_2    X_ts_3      Y_ts
0    id_90     0  0.627422 -0.478959 -0.892121 -0.318527  1.083295
1    id_90     1 -0.448220  0.019303 -0.127440 -0.157412  1.093168
2    id_90     2  0.637957 -0.386629 -0.416992  0.772078  1.026738
3    id_90     3  0.114574 -0.700368  0.415711 -0.584559  0.891013
4    id_90     4  0.318873  0.112828  0.575051  0.149870  0.813476
..     ...   ...       ...       ...       ...       ...       ...
495  id_99    45  0.033853  0.889838  0.867147 -0.226947  0.981591
496  id_99    46 -0.883889  1.506605  0.732750  0.466535  0.846787
497  id_99    47  1.267585 -0.623064  0.034502 -0.162447  0.901342
498  id_99    48 -1.544032  0.310212  1.469414  0.941757  0.912135
499  id_99    49  1.052585 -0.451113  1.395424 -0.229020  0.822956

[500 rows x 7 columns]>


Group the data according to each ```id``` of the time series. 
This is easily done by using the ```groupby()``` method on the DataFrame. This transforms the data into the structure of:

| Group Name (or ID) | Pandas Series   |  
| ------------------ | --------------- |
| id_90              | Series of id_90 |  
| id_91              | Series of id_91 |  
| ...              | ... |  

To receive a certain Series ID ```i``` we just need to call ```df_grouped[i, 0]```.
To receive the corresponding Series data we call ```df_grouped[i, 1]```.

In [9]:
df_grouped = pd.DataFrame(df.groupby('id'))
print(df_grouped.iloc[-1,1])

        id  time    X_ts_0    X_ts_1    X_ts_2    X_ts_3      Y_ts
450  id_99     0  2.715997  0.325934 -0.486907 -2.543788  1.054109
451  id_99     1  1.526065  1.121756 -0.514032 -0.396498  0.764149
452  id_99     2 -0.505807  0.863921  0.798771 -0.444982  1.008917
453  id_99     3  0.788786  0.825285  0.303744 -1.426538  0.986686
454  id_99     4  0.219992 -0.188746 -0.861897 -0.611712  0.816993
455  id_99     5  0.672091 -0.404279  0.618024  0.522364  0.920787
456  id_99     6  0.464268  0.104738 -0.546725  0.383898  0.977540
457  id_99     7  0.548396  0.713342 -0.406693  0.639087  0.967501
458  id_99     8 -0.340459  0.572502 -0.176726 -0.006179  1.087702
459  id_99     9 -0.469286 -0.352744  0.543369  0.351378  1.037535
460  id_99    10  0.238644  0.439631  0.560051 -0.179127  0.892054
461  id_99    11 -1.408751  0.395566 -1.982116  1.018491  0.731176
462  id_99    12 -0.774979  1.008667 -0.172575  0.717566  0.888754
463  id_99    13  0.269323 -1.244125 -0.206261  1.513991  0.81